<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/cdvae_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inverse Design of Next-Generation Superconductors Using Data-Driven Deep Generative Models

Reference: https://pubs.acs.org/doi/10.1021/acs.jpclett.3c01260

In [1]:
%%time
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q pytorch-lightning wandb torchmetrics==0.6.0 pymatgen==2022.4.26
!pip install -q hydra-core jarvis-tools python-dotenv p-tqdm accelerate

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 72.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 98.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# !pip install -q pytorch-lightning==1.3.6

In [3]:
import os
os.chdir('/content')
if not os.path.exists('cdvae'):
  !git clone https://github.com/JARVIS-Materials-Design/cdvae.git

os.chdir('cdvae')
!pip install -e .

Cloning into 'cdvae'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 235 (delta 83), reused 178 (delta 66), pack-reused 0
Receiving objects: 100% (235/235), 146.68 MiB | 21.18 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Obtaining file:///content/cdvae
  Preparing metadata (setup.py) ... done
  Running setup.py develop for cdvae


In [4]:
import os
if not os.path.exists("/content/cdvae/WABDB"):
  os.makedirs("/content/cdvae/WABDB")
if not os.path.exists("/content/cdvae/HYDRA_JOBS"):
  os.makedirs("/content/cdvae/HYDRA_JOBS")
os.environ["PROJECT_ROOT"]="/content/cdvae"
os.environ["WABDB"]="/content/cdvae/WABDB"
os.environ["WABDB_DIR"]="/content/cdvae/WABDB"
os.environ["HYDRA_JOBS"]="/content/cdvae/HYDRA_JOBS"
%env HYDRA_FULL_ERROR=1


env: HYDRA_FULL_ERROR=1


In [5]:
!echo $HYDRA_FULL_ERROR

1


Data was generated using this [script](https://github.com/JARVIS-Materials-Design/cdvae/blob/main/scripts/generate_data_cdvae.py).

In [6]:

import yaml
import pprint

with open('/content/cdvae/conf/train/default.yaml','r') as f:
  yam = yaml.safe_load(f)
#yam.pop('early_stopping')
#yam['pl_trainer']['fast_dev_run']=True
yam['pl_trainer']['gpus']=0

with open('/content/cdvae/conf/train/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

with open('/content/cdvae/conf/optim/default.yaml','r') as f:
  yam = yaml.safe_load(f)

yam['use_lr_scheduler']=False

with open('/content/cdvae/conf/optim/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

{'deterministic': False,
 'early_stopping': {'patience': '${data.early_stopping_patience}',
                    'verbose': False},
 'model_checkpoints': {'save_top_k': 1, 'verbose': False},
 'monitor_metric': 'val_loss',
 'monitor_metric_mode': 'min',
 'pl_trainer': {'accumulate_grad_batches': 1,
                'fast_dev_run': False,
                'gpus': 0,
                'gradient_clip_algorithm': 'value',
                'gradient_clip_val': 0.5,
                'max_epochs': '${data.train_max_epochs}',
                'num_sanity_val_steps': 2,
                'precision': 32,
                'profiler': 'simple'},
 'random_seed': 42}
{'lr_scheduler': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau',
                  'factor': 0.6,
                  'min_lr': '1e-4',
                  'patience': 30},
 'optimizer': {'_target_': 'torch.optim.Adam',
               'betas': [0.9, 0.999],
               'eps': '1e-08',
               'lr': 0.001,
               'weight_de

In [7]:
%%time
import os
os.environ["WANDB_ANONYMOUS"] = "must"
!python cdvae/run.py data=supercon expname=supercon_test02 model.predict_property=True

2023-07-30 13:59:22.492207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 13:59:25.361837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/run.py:164: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'default': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-p

In [8]:
!pip install smact

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.4/130.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.3 MB/s eta 0:00:00


In [9]:
# from datetime import date
# d1 = today.strftime("%Y-%m-%d")

In [10]:
!ls /content/cdvae/HYDRA_JOBS/singlerun/

2023-07-30


Adjust path accordingly

In [11]:
%%time
!python scripts/evaluate.py --n_step_each 5 --num_batches_to_samples 5 --batch_size 5 --model_path "/content/cdvae/HYDRA_JOBS/singlerun/2023-07-30/supercon_test02" --tasks opt gen recon

2023-07-30 16:32:11.267914: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 16:32:12.830334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/pl_data/dataset.py:126: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/cdvae/pl_data/datamodule.py:142: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/

In [12]:
!pip install matminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 35.7 MB/s eta 0:00:00


In [17]:
%%time
!python scripts/compute_metrics.py --root_path "/content/cdvae/HYDRA_JOBS/singlerun/2023-07-30/supercon_test02" --tasks   gen recon

2023-07-30 20:37:36.732817: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 20:37:37.615469: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/pl_data/dataset.py:126: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/cdvae/pl_data/datamodule.py:142: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/

In [42]:
import torch
from jarvis.core.atoms import Atoms
from jarvis.core.atoms import pmg_to_atoms
from jarvis.core.lattice import Lattice
from jarvis.core.specie import atomic_numbers_to_symbols
from jarvis.db.jsonutils import dumpjson
from jarvis.analysis.structure.spacegroup import Spacegroup3D
from collections import Counter
from pymatgen.core.structure import Structure
import pandas as pd
opt_path = "/content/cdvae/HYDRA_JOBS/singlerun/2023-07-30/supercon_test02/eval_opt.pt"
gen_path = "/content/cdvae/HYDRA_JOBS/singlerun/2023-07-30/supercon_test02/eval_gen.pt"
recon_path = "/content/cdvae/HYDRA_JOBS/singlerun/2023-07-30/supercon_test02/eval_recon.pt"
csv_path = "/content/cdvae/data/supercon/test.csv"
df = pd.read_csv(csv_path)
x = torch.load(recon_path)
print(len(df),x["num_atoms"].shape)

105 torch.Size([1, 105])


In [48]:
num_atoms = x["num_atoms"]
atom_types = x["atom_types"]
frac_coords = x["frac_coords"]
lengths = x["lengths"]
angles = x["angles"]
index_list = torch.cumsum(num_atoms[0], dim=0).numpy().tolist()
indice_tuples = []
for i, ii in enumerate(index_list):
    if i == 0:
        tup = [0, index_list[i] - 1]
    else:
        tup = [index_list[i - 1] - 1, index_list[i] - 1]
    indice_tuples.append(tup)

gen_structures = []

for id_needed in range(num_atoms.shape[1]):
    id_fracs = frac_coords[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_atom_types = atom_types[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_lengths = lengths[0].numpy()[id_needed]
    id_angles = angles[0].numpy()[id_needed]
    lat = Lattice.from_parameters(
        id_lengths[0],
        id_lengths[1],
        id_lengths[2],
        id_angles[0],
        id_angles[1],
        id_angles[2],
    ).matrix
    atoms = Atoms(
        lattice_mat=lat,
        elements=atomic_numbers_to_symbols(id_atom_types),
        coords=id_fracs,
        cartesian=False,
    )
    # spg_numb = Spacegroup3D(atoms).space_group_number
    # spg_numbs.append(spg_numb)

    # print()
    # print()
    # print()
    # print("jarvis\n", atoms)
    # struct = Structure(
    #    lattice=Lat.from_parameters(
    #        id_lengths[0],
    #        id_lengths[1],
    #        id_lengths[2],
    #        id_angles[0],
    #        id_angles[1],
    #        id_angles[2],
    #    ),
    #    species=id_atom_types,
    #    coords=id_fracs,
    #    coords_are_cartesian=False,
    # )
    # atoms = pmg_to_atoms(struct)
    # print("pmg\n", atoms)
    # print()
    # print()
    # print()

    # gen_structures.append(atoms.to_dict())
    gen_structures.append(atoms)


In [50]:
test_structures=[]
for i,ii in df.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  test_structures.append(atoms)#.to_dict())

/usr/local/lib/python3.10/dist-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [51]:
for i,j in zip(test_structures,gen_structures):
  print(i.composition.reduced_formula,j.composition.reduced_formula)

Ba2LiSn Ba2Ni
CaCO3 Ca2PO2
Ti3Si CaTi7Si
LiAl3 LiTiAl2
VN YV2N
YB6 LiB5N
Ta2N Ta2NO
MgP2 TaP2
CaSi Ca2SiCl
Li3Sn LiSiSnAu
Be2PtRh LiBe2Pd
NbB ZrBe
FeCo NbFe
YAlS3 FeS3
YPd3 ZrTiPdS
AlVNi2 AlNi2Pd
BeRh Be
SrAlSi BeAlSi
Cr3Ir CaCr7
V3Pt V5CrGe2
TaZn3 VZn2Tc
ScGaNi2 ScZnGaNi
B2SO2 NiBO2
NbMoO4 BO5
NaTiO2 Na2MoO
HfN NO
Y2Fe2Si2C NaHfSiAs
BeCo2P TiBeCoO
POF GePO
NaO2 NaOF
AsP2 YPO
TiNbRe2 Re2TcAs
TiAl Ti
YNbRu2 TiNbGaOs
TiSi SiRu
LiCa2Ga LiCaTiGa
YAl3 Al2VGa
SiCF2 NaAlSiS
HfBeSi Hf2BeSi2F
ScRh3 ScBeRh2
VZn3 ZnRh
NaYF2 VSiSO
BeTc2Rh BeSiTcRu
HfBe2Ni Be3Ni
Li2BeCd LiBe2Ir
BeSiRh2 LiSiRh2
Al2IrRu TaAlIrRh
TiBe2Rh TiBe2Al
TiReN3 Ti2BeN2
ZnMoN2 MoN3
TiPt3 TiZn2Pt
ZrN PdN
PdF2 Na2Zr
HfC HfO
LiP2 HfP
CuBN2 Li2B2N
Sc2CuRu ScCuPdN
Sc2BeRh Sc3Be
Nb3Ru ScNb2Ru
NaTiO2 TiNbO2
BaP2Pb BePO
TaAlNi2 Al2NiP
TaHO TaNiH
HfScOs2 ScOs2O
SrAlGe SrScCu
SrZnSi SrSi2
LiSnCl SrTeCl
B2W BeBSb
GaNF2 AlBF2
BeSiNi2 BeNi2F
Sc2AlRu ScNiAgRu
P2W LiP
Al3F ScAl12P
LiTiS2 AlS3
AlB TiAlBN
Cr2NiS4 AlCr3S3
ScNi4Sn Sc2CrNi3
Co3W C

In [16]:
!pip freeze

absl-py==1.4.0
accelerate==0.21.0
aiohttp==3.8.5
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
antlr4-python3-runtime==4.9.3
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
array-record==0.4.0
arviz==0.15.1
ase==3.22.1
astropy==5.2.2
astunparse==1.6.3
async-timeout==4.0.2
attrs==23.1.0
audioread==3.0.0
autograd==1.6.2
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blinker==1.4
blis==0.7.10
blosc2==2.0.0
bokeh==3.1.1
branca==0.6.0
build==0.10.0
CacheControl==0.13.1
cachetools==5.3.1
catalogue==2.0.9
-e git+https://github.com/JARVIS-Materials-Design/cdvae.git@fe9b01fd01bacb03ff60ba0158f71d52ce29aeb6#egg=cdvae
certifi==2023.7.22
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.6
click-plugins==1.1.1
cligj==0.7.2
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.1.0
cons==0.4.6
contextlib2==21.6.0
contourpy==1.1.0
convertd